In [20]:
import numpy as np
import pandas as pd
import warnings
import statsmodels.api as sm
warnings.filterwarnings("ignore") 

In [21]:
df_price_return = pd.read_csv(r"E:\Desktop\data\TRD_Mnth.csv")
df_factor = pd.read_csv(r"E:\Desktop\data\STK_MKT_THRFACMONTH.csv")

In [22]:
df_price_return.rename(columns={'Stkcd': 'stock_code', 'Trdmnt': 'date', 'Msmvosd': 'market_value', 
                                'Mretnd': 'monthly_return','Markettype':'type'}, inplace=True)
df_factor.rename(columns={'MarkettypeID':'typeid','TradingMonth':'date','RiskPremium1':'risk_prem','HML1':'BM_ratio'},inplace=True)

df_price_return = df_price_return.loc[(df_price_return['type'] == 1) | (df_price_return['type'] == 4) , :]
df_price_return.drop(columns=['type'], inplace=True)
df_factor = df_factor.loc[(df_factor['typeid'] == 'P9706' ) , :]
df_factor.drop(columns=['typeid'], inplace=True)


In [23]:
df_price_return['date'] = pd.to_datetime(df_price_return['date']).dt.to_period('M')
df_factor['date'] = pd.to_datetime(df_factor['date']).dt.to_period('M')
df_price_return

,stock_code,date,market_value,monthly_return
0,1,2000-12,20228171.57,-0.062621
1,1,2001-01,20869008.96,0.031680
2,1,2001-02,19629127.92,-0.059413
3,1,2001-03,22596483.67,0.151171
4,1,2001-04,21635227.58,-0.042540
...,...,...,...,...
718097,605599,2024-08,6300919.57,-0.065274
718098,605599,2024-09,9247778.04,0.107076
718099,605599,2024-10,8283333.57,-0.104289
718100,605599,2024-11,8944444.70,0.079812


In [24]:
df_price_return['last_month_value'] = df_price_return.groupby('stock_code')['market_value'].shift(1)
df_price_return

,stock_code,date,market_value,monthly_return,last_month_value
0,1,2000-12,20228171.57,-0.062621,NaN
1,1,2001-01,20869008.96,0.031680,20228171.57
2,1,2001-02,19629127.92,-0.059413,20869008.96
3,1,2001-03,22596483.67,0.151171,19629127.92
4,1,2001-04,21635227.58,-0.042540,22596483.67
...,...,...,...,...,...
718097,605599,2024-08,6300919.57,-0.065274,6740927.92
718098,605599,2024-09,9247778.04,0.107076,6300919.57
718099,605599,2024-10,8283333.57,-0.104289,9247778.04
718100,605599,2024-11,8944444.70,0.079812,8283333.57


In [25]:
df_price_return = df_price_return.dropna(subset=['last_month_value'])


In [26]:
sorted_df = df_price_return.groupby('date').apply(lambda x: x.sort_values('last_month_value')).reset_index(drop=True)


In [27]:
def sort_val(group):
    group.sort_values('last_month_value', inplace=True)
    return group

sorted_df = sorted_df.groupby('date').apply(sort_val).reset_index(drop=True)
sorted_df['decile'] = sorted_df.groupby('date')['last_month_value'].transform(lambda x: pd.qcut(x, 10, labels=False))
sorted_df['decile'] = sorted_df['decile'] + 1


In [28]:
decile_return = sorted_df.groupby(['date', 'decile'])['monthly_return'].mean().reset_index()
decile_return

,date,decile,monthly_return
0,2001-01,1,-0.040115
1,2001-01,2,-0.020972
2,2001-01,3,-0.011530
3,2001-01,4,0.002192
4,2001-01,5,-0.005961
...,...,...,...
2875,2024-12,6,-0.035134
2876,2024-12,7,-0.036350
2877,2024-12,8,-0.027481
2878,2024-12,9,-0.024363


In [29]:
df1_grouped = df_factor.groupby('date').agg(
    risk_prem=('risk_prem', 'mean'),
    BM_ratio=('BM_ratio', 'mean')
).reset_index()


In [30]:
merged_df = pd.merge(decile_return, df1_grouped, on='date', how='left')
merged_df

,date,decile,monthly_return,risk_prem,BM_ratio
0,2001-01,1,-0.040115,-0.010002,0.028226
1,2001-01,2,-0.020972,-0.010002,0.028226
2,2001-01,3,-0.011530,-0.010002,0.028226
3,2001-01,4,0.002192,-0.010002,0.028226
4,2001-01,5,-0.005961,-0.010002,0.028226
...,...,...,...,...,...
2875,2024-12,6,-0.035134,0.014161,0.038868
2876,2024-12,7,-0.036350,0.014161,0.038868
2877,2024-12,8,-0.027481,0.014161,0.038868
2878,2024-12,9,-0.024363,0.014161,0.038868


In [31]:

results = {}
for decile in range(1, 11):
    decile_data = merged_df[merged_df['decile'] == decile]
    X = decile_data[['risk_prem', 'BM_ratio']]
    X = sm.add_constant(X)  
    y = decile_data['monthly_return']
    model = sm.OLS(y, X).fit()
    results[decile] = model

In [32]:
for decile, model in results.items():
    print(f"Decile {decile}:")
    print(f"Alpha: {model.params['const']:.6f}")
    print(f"Beta1 (Market Risk): {model.params['risk_prem']:.6f}")
    print(f"Beta2 (BM Ratio): {model.params['BM_ratio']:.6f}")
    print(f"R-squared: {model.rsquared:.4f}\n")

Decile 1:
Alpha: 0.024999
Beta1 (Market Risk): 1.021180
Beta2 (BM Ratio): -1.099825
R-squared: 0.5990

Decile 2:
Alpha: 0.014611
Beta1 (Market Risk): 1.027325
Beta2 (BM Ratio): -0.923232
R-squared: 0.6989

Decile 3:
Alpha: 0.011339
Beta1 (Market Risk): 1.032951
Beta2 (BM Ratio): -0.838866
R-squared: 0.7267

Decile 4:
Alpha: 0.008417
Beta1 (Market Risk): 1.056721
Beta2 (BM Ratio): -0.753507
R-squared: 0.7541

Decile 5:
Alpha: 0.006160
Beta1 (Market Risk): 1.038278
Beta2 (BM Ratio): -0.726783
R-squared: 0.7564

Decile 6:
Alpha: 0.005104
Beta1 (Market Risk): 1.041816
Beta2 (BM Ratio): -0.718278
R-squared: 0.7894

Decile 7:
Alpha: 0.004436
Beta1 (Market Risk): 1.060461
Beta2 (BM Ratio): -0.608464
R-squared: 0.8181

Decile 8:
Alpha: 0.003342
Beta1 (Market Risk): 1.054049
Beta2 (BM Ratio): -0.551549
R-squared: 0.8491

Decile 9:
Alpha: 0.003033
Beta1 (Market Risk): 1.048308
Beta2 (BM Ratio): -0.418702
R-squared: 0.8792

Decile 10:
Alpha: 0.001923
Beta1 (Market Risk): 1.050693
Beta2 (BM Ratio)

In [37]:

results_df = pd.DataFrame(columns=[
    'Decile', 
    'Alpha', 
    'Beta_Market_Risk', 
    'Beta_BM_Ratio', 
    'R_squared',
    'Pvalue_Alpha',
    'Pvalue_Market_Risk',
    'Pvalue_BM_Ratio'
])

for decile, model in results.items():
    row_data = {
        'Decile': decile,
        'Alpha': model.params['const'],
        'Beta_Market_Risk': model.params['risk_prem'],
        'Beta_BM_Ratio': model.params['BM_ratio'],
        'R_squared': model.rsquared,
        'Pvalue_Alpha': model.pvalues['const'],
        'Pvalue_Market_Risk': model.pvalues['risk_prem'],
        'Pvalue_BM_Ratio': model.pvalues['BM_ratio']
    }
    results_df = pd.concat(
        [results_df, pd.DataFrame([row_data])], 
        ignore_index=True
    )

results_df = results_df.round({
    'Alpha': 6,
    'Beta_Market_Risk': 6,
    'Beta_BM_Ratio': 6,
    'R_squared': 6,
    'Pvalue_Alpha': 6,
    'Pvalue_Market_Risk': 6,
    'Pvalue_BM_Ratio': 6
})

results_df

,Decile,Alpha,Beta_Market_Risk,Beta_BM_Ratio,R_squared,Pvalue_Alpha,Pvalue_Market_Risk,Pvalue_BM_Ratio
0,1,0.024999,1.021180,-1.099825,0.599018,0.000000,0.0,0.000000
1,2,0.014611,1.027325,-0.923232,0.698891,0.000012,0.0,0.000000
2,3,0.011339,1.032951,-0.838866,0.726671,0.000219,0.0,0.000000
3,4,0.008417,1.056721,-0.753507,0.754133,0.003240,0.0,0.000000
4,5,0.006160,1.038278,-0.726783,0.756383,0.026519,0.0,0.000000
5,6,0.005104,1.041816,-0.718278,0.789390,0.043608,0.0,0.000000
6,7,0.004436,1.060461,-0.608464,0.818127,0.054413,0.0,0.000000
7,8,0.003342,1.054049,-0.551549,0.849079,0.100061,0.0,0.000000
8,9,0.003033,1.048308,-0.418702,0.879235,0.082654,0.0,0.000000
9,10,0.001923,1.050693,-0.034608,0.950128,0.065893,0.0,0.277836


In [38]:
results_df.to_csv("E:\Desktop\data\output.csv", index=False)